In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple
from scipy.optimize import minimize
import sqlite3
from matplotlib.backends.backend_pdf import PdfPages

In [23]:
def get_opening_data(db_path: str) -> pd.DataFrame:
   conn = sqlite3.connect(db_path)
   query = """
   SELECT opening, cp_10, cp_11, cp_12, cp_13, cp_14, avg_opening_cp
   FROM evaluated_openings
   WHERE cp_10 IS NOT NULL 
   AND cp_11 IS NOT NULL
   AND cp_12 IS NOT NULL 
   AND cp_13 IS NOT NULL
   AND cp_14 IS NOT NULL
   """
   df = pd.read_sql_query(query, conn)
   conn.close()
   return df

In [28]:
def analyze_markowitz_portfolio(df: pd.DataFrame, n_top=5):
   # Wybierz najczęstsze otwarcia
   opening_counts = df['opening'].value_counts()
   top_openings = opening_counts.head(n_top)
   
   with PdfPages('markowitz_portfolio_analysis.pdf') as pdf:
       # Strona tytułowa
       plt.figure(figsize=(11.69, 8.27))
       plt.text(0.5, 0.5, f'Analiza Portfelowa Markowitza\nTop {n_top} Najczęstszych Otwarć', 
               horizontalalignment='center', verticalalignment='center', 
               transform=plt.gca().transAxes, fontsize=20)
       plt.axis('off')
       pdf.savefig()
       plt.close()

       # Przygotuj dane zwrotów i macierz kowariancji
       returns_data = {}
       for opening in top_openings.index:
           returns_data[opening] = df[df['opening'] == opening]['avg_opening_cp'].values

       # Oblicz macierz kowariancji
       covariance_matrix = np.zeros((n_top, n_top))
       returns = np.zeros(n_top)
       for i, op1 in enumerate(top_openings.index):
           for j, op2 in enumerate(top_openings.index):
               data1 = returns_data[op1]
               data2 = returns_data[op2]
               min_len = min(len(data1), len(data2))
               covariance_matrix[i,j] = np.cov(data1[:min_len], data2[:min_len])[0,1]
           returns[i] = np.mean(returns_data[op1])

       # Wizualizacja macierzy kowariancji
       plt.figure(figsize=(12, 8))
       plt.imshow(covariance_matrix, cmap='coolwarm')
       plt.colorbar()
       plt.xticks(range(n_top), top_openings.index, rotation=45)
       plt.yticks(range(n_top), top_openings.index)
       plt.title('Macierz Kowariancji')
       plt.tight_layout()
       pdf.savefig()
       plt.close()

       # Oblicz efektywną granicę
       def portfolio_stats(weights):
           portfolio_return = np.sum(weights * returns)
           portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
           return portfolio_return, portfolio_risk

       # Generuj losowe portfele
       n_portfolios = 1000
       all_weights = np.zeros((n_portfolios, n_top))
       all_returns = np.zeros(n_portfolios)
       all_risks = np.zeros(n_portfolios)

       for i in range(n_portfolios):
           weights = np.random.random(n_top)
           weights = weights/np.sum(weights)
           all_weights[i,:] = weights
           portfolio_return, portfolio_risk = portfolio_stats(weights)
           all_returns[i] = portfolio_return
           all_risks[i] = portfolio_risk

       # Znajdź optymalny portfel
       def optimize_portfolio(target_return):
           n = len(returns)
           bounds = tuple((0, 1) for asset in range(n))
           constraints = [
               {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
           ]
           
           result = minimize(lambda x: np.sqrt(np.dot(x.T, np.dot(covariance_matrix, x))),
                           n_top * [1./n_top,],
                           method='SLSQP',
                           bounds=bounds,
                           constraints=constraints)
           return result

       optimal_portfolio = optimize_portfolio(np.mean(returns))
       optimal_weights = optimal_portfolio.x
       optimal_return, optimal_risk = portfolio_stats(optimal_weights)

       # Wizualizacja efektywnej granicy
       plt.figure(figsize=(12, 8))
       plt.scatter(all_risks, all_returns, c='gray', alpha=0.5, label='Możliwe Portfele')
       plt.scatter(optimal_risk, optimal_return, color='red', marker='*', s=200, label='Portfel Optymalny')
       
       for i, opening in enumerate(top_openings.index):
           risk = np.sqrt(covariance_matrix[i,i])
           ret = returns[i]
           plt.scatter(risk, ret, label=f'{opening} (n={top_openings[opening]})')
           
       plt.xlabel('Ryzyko (Odchylenie Standardowe)')
       plt.ylabel('Oczekiwany Zwrot')
       plt.title('Efektywna Granica Markowitza')
       plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
       plt.grid(True)
       plt.tight_layout()
       pdf.savefig()
       plt.close()

       # Podsumowanie wyników
       results = pd.DataFrame({
           'Opening': top_openings.index,
           'Count': top_openings.values,
           'Weight': optimal_weights,
           'Expected Return': returns,
           'Risk': np.sqrt(np.diagonal(covariance_matrix))
       })
       
       return results

In [29]:
def main():
   db_path = 'chess_games.db'
   df = get_opening_data(db_path)
   results = analyze_markowitz_portfolio(df)
   print("\nWyniki analizy portfelowej:")
   print(results)

if __name__ == "__main__":
   main()


Statystyki dla najczęstszych otwarć:
                            opening        risk     reward  count
0                    Modern Defense  224.788628  57.952632     76
1              Van't Kruijs Opening  277.518227  39.288571     70
2                 Caro-Kann Defense  227.047872  17.724138     58
3  French Defense: Knight Variation  253.929132  25.800000     57
4                  Sicilian Defense  218.930771 -16.924528     53
